In [9]:
import time
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurações fixas
origem_url = "vitoria-da-conquista-ba-todos"
destino_url = "sao-paulo-sp-todos"
hoje = datetime.now().date()

# Lista de resultados acumulados
todos_os_resultados = []

# Loop para os próximos 14 dias
for i in range(14):
    print("=" * 60)
    print(f"[{i+1}/14] Iniciando coleta para o dia {hoje + timedelta(days=i)}")

    # Configurações do navegador
    options = Options()
    options.add_argument('--headless')  # rodar em background
    options.add_argument('--window-size=1366,768')
    driver = webdriver.Chrome(options=options)

    data_viagem = hoje + timedelta(days=i)
    data_str = data_viagem.strftime("%Y-%m-%d")
    url = f"https://www.clickbus.com.br/onibus/{origem_url}/{destino_url}?departureDate={data_str}"
    print(f"Acessando: {url}")
    driver.get(url)

    try:
        # Aguarda o carregamento inicial da página
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "time.departure-time"))
        )
        time.sleep(2)

        # Simula scroll para forçar o carregamento de mais resultados
        last_height = driver.execute_script("return document.body.scrollHeight")
        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Captura os elementos da página após carregamento total
        empresas = driver.find_elements(By.CSS_SELECTOR, "div.company")
        horarios = driver.find_elements(By.CSS_SELECTOR, "time.departure-time")
        classes = driver.find_elements(By.CSS_SELECTOR, 'div.service-class')
        precos = driver.find_elements(By.CSS_SELECTOR, "span.price-value")

        total_viagens = min(len(empresas), len(horarios), len(precos))
        print(f"→ Total de viagens encontradas: {total_viagens}")

        for idx in range(total_viagens):
            try:
                empresa = empresas[idx].get_attribute("data-content")
                horario = horarios[idx].text.strip()
                classe = classes[idx].text.strip()
                preco = precos[idx].text.replace("R$", "").strip()  # remove "R$" do preço

                todos_os_resultados.append({
                    "data_viagem": data_str,
                    "empresa": empresa,
                    "horario_saida": horario,
                    "classe": classe,
                    "preco": preco
                })
            except Exception as e:
                print(f"[ERRO] Falha ao extrair dados da viagem {idx + 1}: {e}")

    except Exception as e:
        print(f"[ERRO] Falha no carregamento dos elementos: {e}")
        with open(f"erro_viagem_{data_str}.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[INFO] HTML salvo em erro_viagem_{data_str}.html")

    driver.quit()
    time.sleep(2)

# Criação do DataFrame com todos os dados
df_resultado = pd.DataFrame(todos_os_resultados)

# Exibe preview do resultado no console
print(df_resultado)

# Exportação para Excel
df_resultado.to_excel("passagens_clickbus.xlsx", index=False)

[1/14] Iniciando coleta para o dia 2025-08-03
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-03
→ Total de viagens encontradas: 1
[2/14] Iniciando coleta para o dia 2025-08-04
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-04
→ Total de viagens encontradas: 13
[3/14] Iniciando coleta para o dia 2025-08-05
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-05
→ Total de viagens encontradas: 21
[4/14] Iniciando coleta para o dia 2025-08-06
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-06
→ Total de viagens encontradas: 20
[5/14] Iniciando coleta para o dia 2025-08-07
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-07
→ Total de viagens encontradas: